# ETL_CLEANING

### DOMAIN: ECOMMERCE | INDIAN HERBAL SKIN/HAIR CARE PRODUCTS

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

### Logging setup for audit

In [2]:
logging.basicConfig(filename = "etl_log.txt", level = logging.INFO, format = "%(asctime)s - %(levelname)s - %(message)s")
logging.info("ECOMMERCE ETL PIPELINE STARTED")
logging.info("="*60)

### 1. Load Raw Data

In [3]:
def concat_orders(file_paths):
    dfs = []
    for path in file_paths:
        if not os.path.exists(path):
            logging.error(f"Files missing: {path}")
            raise FileNotFoundError("File not found")
        df = pd.read_csv(path)
        dfs.append(df)
        logging.info(f"Loaded {path}: {df.shape}")
    orders_raw = pd.concat(dfs, ignore_index=True, sort=False)
    logging.info(f"Combined orders: {orders_raw.shape}")
    return orders_raw

In [4]:

def load_data(order_list, product, customer):
        try:
            
            #Orders
            orders_raw = concat_orders(order_list)
            orders_raw.to_csv("orders_raw.csv", index = False)
            logging.info(f"Orders combined: {orders_raw.shape}")
    
            #products
            products_raw = pd.read_csv(product)
        
            #customers
            customers_raw = pd.read_csv(customer)

            return orders_raw, products_raw, customers_raw
            
        except FileNotFoundError as e:
            logging.error(f"File missing: {e}")
            raise
       

In [5]:
order_list = [
    "orders_nov_raw.csv",
    "orders_dec_raw.csv"
]

orders_raw, products_raw, customers_raw = load_data(
    order_list,
    "products_raw.csv",
    "customers_raw.csv"
)

### 2. Data Quality Assesment

In [6]:
orders_raw.info()
products_raw.info()
customers_raw.info()

missing_orders = orders_raw.isnull().sum()
missing_products = products_raw.isnull().sum()
missing_customers = customers_raw.isnull().sum()

logging.info(f"Missing values in orders_raw:\n{missing_orders}")
logging.info(f"Missing values in products_raw:\n{missing_products}")
logging.info(f"Missing values in customers_raw:\n{missing_customers}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Order ID              600 non-null    int64  
 1   Order Name            600 non-null    object 
 2   Created at            600 non-null    object 
 3   Financial Status      600 non-null    object 
 4   Customer ID           600 non-null    int64  
 5   Total Price           600 non-null    float64
 6   Subtotal Price        600 non-null    int64  
 7   Total Discounts       600 non-null    int64  
 8   Shipping Cost         600 non-null    int64  
 9   Total Tax             600 non-null    float64
 10  Lineitem quantity     600 non-null    int64  
 11  Lineitem sku          600 non-null    object 
 12  Lineitem price        600 non-null    int64  
 13  Lineitem total price  600 non-null    int64  
 14  Lineitem name         600 non-null    object 
 15  Refund ID             6

### Filling missing values

In [7]:
orders_raw['Refund ID'] = orders_raw['Refund ID'].fillna(0)

### 3. Handle duplicates and cancelled orders

In [8]:
orders_raw.drop_duplicates(["Order ID"], inplace=True)

In [9]:
orders_raw.duplicated().sum()

np.int64(0)

In [10]:
orders = orders_raw.copy()
orders = orders[~orders['Financial Status'].isin(['cancelled', 'refunded'])]
logging.info(f"After dedupe/cancelled: {len(orders):,}")

In [11]:
orders

,Order ID,Order Name,Created at,Financial Status,Customer ID,Total Price,Subtotal Price,Total Discounts,Shipping Cost,Total Tax,Lineitem quantity,Lineitem sku,Lineitem price,Lineitem total price,Lineitem name,Refund ID,Risk Level
0,110460500,#ORD110460500,2025-11-11,pending,1220,1112.0,900,0,50,162.0,5,HERB002,180,900,Neem Face Wash,0,high
1,110460501,#ORD110460501,2025-11-29,paid,1116,1436.0,1200,30,50,216.0,4,HERB003,300,1200,Tulsi Shampoo,0,medium
2,110460502,#ORD110460502,2025-11-15,paid,1376,1426.0,1200,20,30,216.0,4,HERB003,300,1200,Tulsi Shampoo,0,medium
3,110460503,#ORD110460503,2025-11-05,paid,1165,1082.0,900,50,70,162.0,5,HERB002,180,900,Neem Face Wash,0,medium
4,110460504,#ORD110460504,2025-11-06,paid,1379,279.6,220,30,50,39.6,1,HERB005,220,220,Aloe Vera Gel,0,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,110460492,#ORD60492,2025-12-16,paid,1380,444.8,360,30,50,64.8,2,HERB002,180,360,Neem Face Wash,0,medium
593,110460493,#ORD60493,2025-12-04,paid,1468,1062.0,900,50,50,162.0,5,HERB002,180,900,Neem Face Wash,0,low
594,110460494,#ORD60494,2025-12-12,paid,1436,1485.0,1250,20,30,225.0,5,HERB001,250,1250,Amla Hair Oil,0,high
598,110460498,#ORD60498,2025-12-04,paid,1153,444.8,360,50,70,64.8,2,HERB002,180,360,Neem Face Wash,0,low


### 4. Datatype Conversion

In [12]:
date = ['Created at']
for col in date:
    if col in orders.columns:
        orders[col] = pd.to_datetime(orders[col], errors='coerce')

In [13]:
numeric = ['Total Price', 'Subtotal Price', 'Total Discounts', 'Shipping Cost', 
                'Total Tax', 'Lineitem quantity', 'Lineitem price', 'Lineitem total price']
for col in numeric:
    if col in orders.columns:
        orders[col] = pd.to_numeric(orders[col], errors='coerce').fillna(0)

In [14]:
ID = ['Customer ID', 'Order ID']
for col in ID:
    if col in orders.columns:
        orders[col] = pd.to_numeric(orders[col], errors='coerce')

In [15]:
orders[['Created at', 'Total Price', 'Customer ID']].dtypes

Created at     datetime64[ns]
Total Price           float64
Customer ID             int64
dtype: object

### 5. Business Logic

In [16]:
orders['Net Revenue'] = orders['Total Price'] - orders['Total Discounts']

In [17]:
orders['Fulfillment Cost'] = orders['Shipping Cost'] + orders['Total Tax']

In [18]:
orders['Contribution Margin'] = orders['Net Revenue'] - orders['Shipping Cost']

#Time features

In [19]:
orders['Order Year'] = orders['Created at'].dt.year

In [20]:
orders['Order Month'] = orders['Created at'].dt.month

In [21]:
orders['Order week'] = orders['Created at'].dt.isocalendar().week

In [22]:
orders['Days Since Order'] = (datetime.now() - orders['Created at']).dt.days

In [23]:
orders['First Purchase'] = orders.groupby('Customer ID')['Created at'].transform('min')

### 6. Product join

In [24]:
if products_raw is not None and 'Variant SKU' in products_raw.columns:
    orders['Lineitem sku'] = orders['Lineitem sku'].str.strip().str.upper()
    products_raw['Variant SKU'] = products_raw['Variant SKU'].str.strip().str.upper()
    orders = orders.merge(products_raw[['Variant SKU', 'Cost per item']], left_on = 'Lineitem sku', right_on = 'Variant SKU', how='left')

### 7. Margin Calculation

In [25]:
orders['Cost per item'] = pd.to_numeric(orders['Cost per item'], errors='coerce').fillna(0)

In [26]:
orders['Lineitem COGS'] = orders['Cost per item'] * orders['Lineitem quantity']

In [27]:
orders['Lineitem Gross Profit'] = orders['Lineitem total price'] - orders['Lineitem COGS']

In [28]:
orders['Gross Margin %'] = (orders['Lineitem Gross Profit'] / orders['Lineitem total price']) * 100

### 8. RFM Preparation

In [29]:
rfm_base = orders.groupby('Customer ID').agg({
    'Created at': ['count', 'max', 'first'],
    'Net Revenue': 'sum',
    'Order ID': 'nunique'
}).round(2).reset_index()

In [30]:
rfm_base.columns = ['Customer ID', 'Frequency', 'Recency', 'First Order Date', 
                   'Monetary Value', 'Order Count']

In [31]:
rfm_base['Recency Days'] = (datetime.now() - rfm_base['Recency']).dt.days

In [32]:
rfm_base['Tenure Days'] = (datetime.now() - rfm_base['First Order Date']).dt.days

In [33]:
logging.info(rfm_base[['Recency Days', 'Frequency', 'Monetary Value']].describe())

In [34]:
master_orders = orders[['Order ID', 'Customer ID', 'Created at', 'Financial Status',
                       'Net Revenue', 'Contribution Margin', 'Lineitem sku', 
                       'Lineitem quantity', 'Lineitem total price', 'Gross Margin %',
                       'Order Year', 'Order Month', 'Days Since Order']].copy()

In [35]:
master_orders.to_csv('master_orders.csv', index=False)
orders.to_csv('orders.csv', index=False)
rfm_base.to_csv('rfm.csv', index=False)

In [36]:
summary_stats = pd.DataFrame({
    'Metric': ['Total Orders', 'Total Revenue', 'Avg Order Value', 'Total Customers'],
    'Value': [len(orders), orders['Net Revenue'].sum(), 
              orders['Net Revenue'].mean(), len(rfm_base)]
}).round(2)
logging.info(summary_stats)

In [37]:
logging.info(f"ETL SUCCESS | Revenue: {orders['Net Revenue'].sum():,.0f} | Customers: {len(rfm_base):,}")